In [1]:
import sympy as sp
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)
from sympy.physics.mechanics import dynamicsymbols, Point, ReferenceFrame
import xml.etree.ElementTree as ET
tree = ET.parse('model1.xml')
root = tree.getroot()
from lxml.etree import fromstring
theta1, theta2, theta3 = dynamicsymbols('theta1 theta2 theta3')
links = []
joints = []
for item in root.findall('./model/'):
    if item.tag == 'joint':
        joints.append(item.attrib['name'])
for item in root.findall('./model/'):
    if item.tag == 'link':
        links.append(item.attrib['name'])
keys = ['x','y','z','roll','pitch','yaw']
values = []
for x in joints:
    for item in root.findall('./model/'):
        if item.attrib['name'] == x:
            for child in item:
                if child.tag == 'pose':
                    values.append(child.text.split())
linkvalues = []
for x in links:
    for item in root.findall('./model/'):
        if item.attrib['name'] == x:
            for child in item:
                if child.tag == 'pose':
                    linkvalues.append(child.text.split())
linkposes = dict(zip(links, map(list, (zip(*linkvalues)))))
lin = dict(zip(links, linkvalues))
jointposes = dict(zip(joints, values))
referenceframelist = []
i = 0
for key in lin:
    key = ReferenceFrame(key.split()[0])
    i +=1
    referenceframelist.append(key)
a = 0
referenceframelist2 = []
for key in jointposes:
    key = ReferenceFrame(key.split()[0])
    a +=1
    referenceframelist2.append(key)

base_frame = ReferenceFrame('base')
referenceframelist2.insert(0, base_frame)

print(referenceframelist)
print(referenceframelist2)

for x in jointposes:
    for i in range(6):
        jointposes[x][i] = float(jointposes[x][i])

jointangles = []
a = 1
for item in joints:
    jointangles.append(('theta' + str(a)))
    a = a+1


jointangletheta = []
for value in jointangles:
    value = dynamicsymbols(value)
    jointangletheta.append(value)


axisvalues = []
for x in joints:
    for item in root.findall('./model/'):
        if item.attrib['name'] == x:
            for child in item:
                for ax in child:
                    if child.tag == 'axis':
                        axisvalues.append(ax.text.split())
jointaxis = dict(zip(joints, axisvalues))

axval = -1
print(referenceframelist2)
for x in referenceframelist2:
    index = referenceframelist2.index(x)
    if referenceframelist2.index(x) == 0:
        pass
    else:
        if jointaxis[str(x)][0] == '1': 
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3] + jointangletheta[axval], jointposes[str(x)][4], jointposes[str(x)][5]], 123)
        elif jointaxis[str(x)][1] == '1':
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3], jointposes[str(x)][4] + jointangletheta[axval], jointposes[str(x)][5]], 123)
        elif jointaxis[str(x)][2] == '1':
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3], jointposes[str(x)][4] , jointposes[str(x)][5] + jointangletheta[axval]], 123)
        else:
            x.orient(referenceframelist2[index-1], 'Body', [jointposes[str(x)][3], jointposes[str(x)][4] , jointposes[str(x)][5] + jointangletheta[axval]], 123)
        axval = axval + 1

        rotationmatrix = []

for x in referenceframelist2:
    ind = referenceframelist2.index(x)
    if referenceframelist2.index(x) == len(referenceframelist2)-1:
        pass
    else:
        rotmatrix = x.dcm(referenceframelist2[ind+1])
        rotationmatrix.append(rotmatrix.simplify())

[base, link1, link2]
[base, shoulder, joint]
[base, shoulder, joint]


In [2]:
from sympy import *
init_printing(use_unicode=True)
transformationmatrix = []
i = 1
for x in rotationmatrix:
    r1 = [x[0], x[1], x[2], jointposes[str(referenceframelist2[i])][0]]
    r2 = [x[3], x[4], x[5], jointposes[str(referenceframelist2[i])][1]]
    r3 = [x[6], x[7], x[8], jointposes[str(referenceframelist2[i])][2]]
    r4 = [0, 0, 0, 1]
    
    i = i+1
    tm = Matrix([r1, r2, r3, r4])
    transformationmatrix.append(tm)
transformationmatrix

⎡⎡cos(θ₂(t))  -sin(θ₂(t))  0  0.0⎤  ⎡            -0.416146836547142⋅cos(θ₁(t) 
⎢⎢                               ⎥  ⎢                                         
⎢⎢sin(θ₂(t))  cos(θ₂(t))   0  0.0⎥  ⎢-0.642872836134547⋅sin(θ₁(t)) - 0.6812427
⎢⎢                               ⎥, ⎢                                         
⎢⎢    0            0       1  0.0⎥  ⎢-0.763718336650279⋅sin(θ₁(t)) + 0.6051272
⎢⎢                               ⎥  ⎢                                         
⎣⎣    0            0       0   1 ⎦  ⎣                             0           

+ 3.0)                            0.416146836547142⋅sin(θ₁(t) + 3.0)          
                                                                              
20256403⋅cos(θ₁(t))  0.681242720256403⋅sin(θ₁(t)) - 0.642872836134547⋅cos(θ₁(t
                                                                              
47241369⋅cos(θ₁(t))  -0.605127247241369⋅sin(θ₁(t)) - 0.763718336650279⋅cos(θ₁(
                                                   

In [3]:
sub1 = transformationmatrix[0].subs({jointangletheta[0]:pi, jointangletheta[1]:pi/4})
sub2 = transformationmatrix[1].subs({jointangletheta[0]:pi, jointangletheta[1]:pi/6})

In [4]:
transform2 = [sub1, sub2]
inversedjointtransform = []
ind = 1
for x in transform2:
    if transform2.index(x) == 0:
        inversedjointtransform.append(x)
    else:
        inversedjointtransform.append(transform2[ind].inv())
        ind = ind+1

In [5]:
from functools import reduce
reduce(lambda x, y: x*y, inversedjointtransform)

⎡-0.176627800369031⋅√2  0.0191849420609282⋅√2  -0.684422791945824⋅√2  0.529883
⎢                                                                             
⎢-0.235354445296652⋅√2  0.662057778195475⋅√2   0.0792955447044552⋅√2  0.706063
⎢                                                                             
⎢  0.909297426825682      0.350175488374015     -0.224845095366153     -2.7278
⎢                                                                             
⎣          0                      0                      0                    

401107093⋅√2⎤
            ⎥
335889956⋅√2⎥
            ⎥
9228047704  ⎥
            ⎥
 1          ⎦